In [2]:
%load_ext autoreload
%autoreload 2

import sys
sys.path.append("../")

from sae.sparse_autoencoder import *
from sae.activation_store import *
from sae.train import ModelTrainer
from sae.config import create_config, Config
from sae.metrics import *
from sae.utils import get_blog_checkpoint, get_blog_sparsity, create_lineplot_histogram
from unlearning.metrics import *
from unlearning.metrics import calculate_wmdp_bio_metrics

from transformer_lens import HookedTransformer, utils
from sae.metrics import compute_metrics_post_by_text

import plotly.express as px
import plotly.graph_objs as go
from datasets import load_dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from transformer_lens import HookedTransformer
import time
torch.set_grad_enabled(False)

In [3]:

def get_loss_added(model,
                   activation_store,
                   n_batch=2):
    
    activation_store.iterable_dataset = iter(activation_store.dataset)
    
    with torch.no_grad():
        loss_diffs = []
        per_token_loss_diffs = []
        token_list = []
        
        for _ in tqdm(range(n_batch)):
            
            tokens = activation_store.get_batch_tokenized_data()

            logits = model(tokens, return_type="logits")
            loss_per_token = get_per_token_loss(logits, tokens)

            per_token_loss_diff = loss_per_token

            per_token_loss_diffs.append(per_token_loss_diff)
            token_list.append(tokens)
        
        return torch.vstack(per_token_loss_diffs), torch.vstack(token_list)
    
def get_per_token_loss(logits, tokens):
    log_probs = F.log_softmax(logits, dim=-1)
    # Use torch.gather to find the log probs of the correct tokens
    # Offsets needed because we're predicting the NEXT token (this means the final logit is meaningless)
    # None and [..., 0] needed because the tensor used in gather must have the same rank.
    predicted_log_probs = log_probs[..., :-1, :].gather(dim=-1, index=tokens[..., 1:, None])[..., 0]
    return -predicted_log_probs
    
def get_loss_added_rmu_model(rmu_model,
                             base_model,
                             activation_store,
                             n_batch=2):
    
    activation_store.iterable_dataset = iter(activation_store.dataset)
    
    with torch.no_grad():
        loss_diffs = []
        per_token_loss_diffs = []
        token_list = []
        
        for _ in tqdm(range(n_batch)):
            
            tokens = activation_store.get_batch_tokenized_data()

            base_logits = base_model(tokens, return_type="logits")
            base_loss_per_token = get_per_token_loss(base_logits, tokens)
            # base_loss = base_model(tokens, return_type="loss")
            # base_loss_per_token = utils.lm_cross_entropy_loss(base_logits, tokens, per_token=True)

            rmu_logits = rmu_model(tokens, return_type="logits")
            rmu_loss_per_token = get_per_token_loss(rmu_logits, tokens)
            # rmu_loss = rmu_model(tokens, return_type="loss")
            # rmu_loss_per_token = utils.lm_cross_entropy_loss(rmu_logits, tokens, per_token=True)

            loss_diff = rmu_loss_per_token.mean() -  base_loss_per_token.mean()
            per_token_loss_diff = rmu_loss_per_token - base_loss_per_token

            loss_diffs.append(loss_diff)
            per_token_loss_diffs.append(per_token_loss_diff)
            token_list.append(tokens)
        
        return torch.tensor(loss_diffs), torch.vstack(per_token_loss_diffs), torch.vstack(token_list)

In [4]:
print("starting")

tlens_model_name = "google/gemma-2b-it"
base_model_name = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(base_model_name)

tmodel = AutoModelForCausalLM.from_pretrained(base_model_name, torch_dtype='auto') #.to("cuda")
base_model = HookedTransformer.from_pretrained(tlens_model_name, hf_model=tmodel)

print("done")

starting


`config.hidden_act` is ignored, you should use `config.hidden_activation` instead.
Gemma's activation function will be set to `gelu_pytorch_tanh`. Please, use
`config.hidden_activation` if you want to override this behaviour.
See https://github.com/huggingface/transformers/pull/29402 for more details.


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer
done


In [5]:
from huggingface_hub import hf_hub_download

# resid pre 9
REPO_ID = "eoinf/unlearning_saes"
FILENAME = "jolly-dream-40/sparse_autoencoder_gemma-2b-it_blocks.9.hook_resid_pre_s16384_127995904.pt"

filename = hf_hub_download(repo_id=REPO_ID, filename=FILENAME)
sae = load_saved_sae(filename)

activation_store = ActivationStoreAnalysis(sae.cfg, base_model)


buffer
dataloader


In [7]:
metrics_list = []
loss_added_list = []

tlens_model_name = "google/gemma-2b-it"
model_names = ['gemma_2b_it_rmu_6', 'gemma_2b_it_rmu_10', 'gemma_2b_it_rmu_30', 'gemma_2b_it_rmu_60', 'gemma_2b_it_rmu_100']

for model_name in model_names:

    hf_model_name = "eoinf/" + model_name
    tmodel = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto') #.to("cuda")

    rmu_model = HookedTransformer.from_pretrained(tlens_model_name, hf_model=tmodel, tokenizer=tokenizer)
    
    metrics = calculate_wmdp_bio_metrics(rmu_model,
                                        question_subset=None,
                                        question_subset_file="../data/wmdp-bio_gemma_2b_it_correct.csv",
                                        permutations=None)
    
    metrics_list.append(metrics)
    
    print("done metrics")
    
    loss_added = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=20)
    
    loss_added_list.append(loss_added)
    print("done", model_name)
    
print("done")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 29/29 [00:06<00:00,  4.29it/s]


done metrics


100%|██████████| 20/20 [00:23<00:00,  1.19s/it]


...

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 29/29 [00:06<00:00,  4.29it/s]


done metrics


100%|██████████| 20/20 [00:24<00:00,  1.20s/it]


done gemma_2b_it_rmu_100
done


In [8]:
[x[0].mean().item() for x in loss_added_list]

[-0.006862831301987171,
 -0.006507921032607555,
 -0.0032344937790185213,
 -0.0025219798553735018,
 0.24644620716571808]

In [12]:
[x['mean_correct'] for x in metrics_list]

[0.9941860437393188,
 0.9941860437393188,
 0.6104651093482971,
 0.39534884691238403,
 0.36627906560897827]

In [13]:
[x['mean_predicted_prob_of_correct_answers'] for x in metrics_list]

[0.9896829128265381,
 0.9899671077728271,
 0.6090828776359558,
 0.3412681818008423,
 0.26022282242774963]

In [15]:
metrics_list2 = []
loss_added_list2 = []

tlens_model_name = "google/gemma-2b-it"
model_names = ['gemma_2b_it_rmu_60']

for model_name in model_names:

    hf_model_name = "eoinf/" + model_name
    tmodel = AutoModelForCausalLM.from_pretrained(hf_model_name, torch_dtype='auto') #.to("cuda")

    rmu_model = HookedTransformer.from_pretrained(tlens_model_name, hf_model=tmodel, tokenizer=tokenizer)
    
    metrics = calculate_wmdp_bio_metrics(rmu_model,
                                        question_subset=None,
                                        question_subset_file="../data/wmdp-bio_gemma_2b_it_correct.csv",
                                        permutations=all_permutations)
    
    metrics_list2.append(metrics)
    
    print("done metrics")
    
    loss_added = get_loss_added_rmu_model(rmu_model, base_model, activation_store, n_batch=20)
    
    loss_added_list2.append(loss_added)
    print("done", model_name)
    
print("done")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loaded pretrained model google/gemma-2b-it into HookedTransformer


100%|██████████| 688/688 [02:17<00:00,  4.99it/s]


done metrics


100%|██████████| 20/20 [00:24<00:00,  1.22s/it]


done gemma_2b_it_rmu_60
done


In [16]:
metrics_list2[0]

{'mean_correct': 0.39534884691238403,
 'total_correct': 1632,
 'is_correct': array([0., 1., 0., ..., 0., 0., 0.], dtype=float32),
 'output_probs': array([[2.1784686e-01, 6.4719208e-03, 6.2269843e-01, 1.4312220e-01],
        [2.3252216e-01, 4.2762435e-03, 6.5412521e-01, 9.7980998e-02],
        [2.3325740e-01, 7.9972176e-03, 6.0639840e-01, 1.4238898e-01],
        ...,
        [9.9766374e-01, 1.1716344e-06, 1.1497475e-05, 2.0278682e-07],
        [9.9772125e-01, 4.6575028e-06, 7.3904243e-07, 1.6371770e-07],
        [9.9780995e-01, 3.0797348e-06, 4.3868951e-07, 8.7680355e-08]],
       dtype=float32),
 'actual_answers': array([3, 2, 3, ..., 2, 1, 1]),
 'predicted_answers': array([2, 2, 2, ..., 0, 0, 0]),
 'predicted_probs': array([0.6226984 , 0.6541252 , 0.6063984 , ..., 0.99766374, 0.99772125,
        0.99780995], dtype=float32),
 'predicted_probs_of_correct_answers': array([1.4312220e-01, 6.5412521e-01, 1.4238898e-01, ..., 1.1497475e-05,
        4.6575028e-06, 3.0797348e-06], dtype=float32

In [26]:
(metrics_list2[0]['is_correct'].reshape(-1, 24).mean(axis=1) == 1).astype(float).sum()/172

0.14534883720930233

In [14]:
from itertools import permutations
all_permutations = list(permutations([0, 1, 2, 3]))

In [ ]:
# Load model directly


# model_name = "HuggingFaceH4/zephyr-7b-beta"
# model_name = "../wmdp/models/zephyr_rmu_30"
model_name = "../wmdp/models/gemma_2b_it_rmu_30"
# model_name = "google/gemma-2b-it"

tokenizer = AutoTokenizer.from_pretrained(model_name)
tmodel = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype='auto') #.to("cuda")

tlens_model_name = "mistralai/Mistral-7B-v0.1"
tlens_model_name = "google/gemma-2b-it"
print("starting")
rmu_model = HookedTransformer.from_pretrained(tlens_model_name, hf_model=tmodel, tokenizer=tokenizer)
print("done")